In [3]:
import sys
import SimpleITK as sitk
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline 
import numpy as np
import os
import fnmatch
import shutil
from ipywidgets import interact, fixed
import pandas as pd
import warnings
from skimage import exposure
from skimage import data
from skimage.transform import rotate
import _pickle as cPickle
import random


def myshow(image_arr,slice_n):
    plt.imshow(image_arr[slice_n,:,:],cmap='gray')
    plt.show()

In [4]:
#reading corresponding dataframes with arrays 

t2_df=pd.read_pickle('./t2_augmented.pkl')
adc_df=pd.read_pickle('./adc_augmented.pkl')
bval_df=pd.read_pickle('./bval_augmented.pkl')
ktrans_df=pd.read_pickle('./ktrans_augmented.pkl')

In [5]:
#function to get zero-mean unit variance 

def get_optimized(data):
    
    for i in range(len(data)):
        array = data.loc[i, 'patches']
        optimized = (array - array.mean(axis=0)) / array.std(axis=0)
        data.at[i, 'patches']=optimized
    return data


t2_o = get_optimized(t2_df)    
adc_o = get_optimized(adc_df)
bval_o = get_optimized(bval_df)
ktrans_o = get_optimized(ktrans_df)


/home/andrii/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [10]:
#once we get adequately balanced datasets we can right them top the disk fopr further analysis




t2_o.to_pickle('./t2_optimized.pkl')
adc_o.to_pickle('./adc_optimized.pkl')
bval_o.to_pickle('./bval_optimized.pkl')
ktrans_o.to_pickle('./ktrans_optimized.pkl')